In [1]:
import pandas as pd
df = pd.read_csv('/home/xuemeng/work_xuemeng/soc/SoilHealthDataCube/landsat_based_spectral_indices/data/012_test_trend.point.csv')

# df = pd.read_parquet('/home/xuemeng/work_xuemeng/trend-with-uncertainty/soil_organized.pq')
# prop = 'bulk.density.fe'
# df.loc[df[f'{prop}_qa']>2, prop].max()

In [2]:
df = df[['point_id', 'year', 'nuts2', 'gps_lat', 'gps_long', 'lc1_label','lu1_label']]
df = df.rename(columns={'point_id':'id','gps_long':'lon','gps_lat':'lat','lc1_label':'lc1','lu1_label':'lu1'})

### the model training is done in SOC EU test

In [3]:
ylist = list(range(2000,2023,1))
for ii in [2009,2012,2015,2018]:
    ylist.remove(ii)
    
for iid in df['id'].unique():
    for year in ylist:
        df.loc[len(df)] = [iid, year, None, None, None, None, None]
        
        
df['lat'] = df.groupby('id')['lat'].transform(lambda x: x.fillna(x.mean()))
df['lon'] = df.groupby('id')['lon'].transform(lambda x: x.fillna(x.mean()))
df['nuts2'] = df.groupby('id')['nuts2'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

years = df['year'].unique().tolist()
years = [int(ii) for ii in years]

In [4]:
from shapely.geometry import Point
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
print('data size before overlay', df.shape)
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
df = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

data size before overlay (69, 7)


In [5]:
from skmap.catalog import DataCatalog
from skmap.loader import TiledDataLoader
from skmap.overlay import SpaceOverlay, SpaceTimeOverlay
from skmap.misc import find_files, GoogleSheet, ttprint
from osgeo.gdal import BuildVRT, SetConfigOption
import time

base_path = [f'http://192.168.1.{gaia_id}:8333' for gaia_id in range(30,47)]
GDAL_OPTS = {'GDAL_HTTP_VERSION': '1.0', 'CPL_VSIL_CURL_ALLOWED_EXTENSIONS': '.tif'}
max_ram_mb = 750000
n_threads = 96

# read in gsheet
gsheet_key = '/home/xuemeng/work_xuemeng/trend-with-uncertainty/gaia-319808-913d36b5fca4.json'
gsheet_url = 'https://docs.google.com/spreadsheets/d/1lNTpzdHBG5dirYj46iBDRJMk_YAV0Um2ovBc8v3dR9w/edit?gid=78425683#gid=78425683'
gsheet = GoogleSheet(gsheet_key, gsheet_url, verbose=False)

# create catalog
catalog = DataCatalog.create_catalog(catalog_def=gsheet.eu_soil_prop, years=years, base_path=base_path)
# catalog.save_json('soil.prop_eu.json')



Year 2000 not available for layer wv_mcd19a2v061_n_1km_s_YYYY0101_YYYY0131_go_epsg.4326_v20230619, propagating year 2001
Year 2000 not available for layer non.photosynthetic.veg_mcd43a4.fc_m_500m_s_YYYY0101_YYYY1231_go_epsg.4326_v20240616, propagating year 2001
Year 2000 not available for layer non.photosynthetic.veg_mcd43a4.fc_mx_500m_s_YYYY0101_YYYY1231_go_epsg.4326_v20240616, propagating year 2001
Year 2000 not available for layer non.photosynthetic.veg_mcd43a4.fc_std_500m_s_YYYY0101_YYYY1231_go_epsg.4326_v20240616, propagating year 2001
Year 2000 not available for layer photosynthetic.veg_mcd43a4.fc_m_500m_s_YYYY0101_YYYY1231_go_epsg.4326_v20240616, propagating year 2001
Year 2000 not available for layer photosynthetic.veg_mcd43a4.fc_mx_500m_s_YYYY0101_YYYY1231_go_epsg.4326_v20240616, propagating year 2001
Year 2000 not available for layer photosynthetic.veg_mcd43a4.fc_std_500m_s_YYYY0101_YYYY1231_go_epsg.4326_v20240616, propagating year 2001
Year 2000 not available for layer wv_mc

In [6]:
start = time.time()
space_overlay = SpaceTimeOverlay(
        col_date='year',
        points=df, 
        catalog=catalog,
        raster_tiles='ard2_final_status.gpkg',
        verbose=True,
        n_threads=n_threads,
        tile_id_col='TILE')

print(f"Extraction of overlay meta-data: {(time.time() - start):.2f} s")


[16:30:05] Overlay 3 points from 2000 in 598 raster layers
[16:30:05] Reading ard2_final_status.gpkg
[16:31:38] Scanning blocks of 598 layers
[16:31:38] Finding query pixels for 0eedd36ec93fd9c99387cc8f5c320801 (34 layers)
[16:31:38] Finding query pixels for 1111cac5fd6669660413a5daaf395e08 (11 layers)
[16:31:38] Finding query pixels for 20b4790b3a160a2ac8adefedfe07fbe7 (1 layers)
[16:31:38] Finding query pixels for 2bff5e510f1e47be04d69728dd453b6b (123 layers)
[16:31:38] Finding query pixels for 2fbfe950e4c22a02ba7e3b52884be34f (3 layers)
[16:31:38] Finding query pixels for 435d45ba442271d360c4ea7ec0c92bc6 (1 layers)
[16:31:39] Finding query pixels for 4febf6577f1f162c52ad482e075d0c76 (7 layers)
[16:31:39] Finding query pixels for 56140668da1ff98e935fb06418a942f2 (1 layers)
[16:31:40] Finding query pixels for 660960b8a39c6afeae9171be064ae057 (139 layers)
[16:31:40] Finding query pixels for 67d1c8ae1f1ff92ae4147038afb08e2b (10 layers)
[16:31:41] Finding query pixels for 6df3e8a271732db

In [7]:
start = time.time()
ovelayed_data = space_overlay.run(gdal_opts=GDAL_OPTS, max_ram_mb=max_ram_mb, out_file_name="trend_overlaid.pq")
print(f"Reading overlayed layers: {(time.time() - start):.2f} s")
print(f'data size: ', ovelayed_data.shape)

[16:56:31] Running the overlay for 2000
[16:56:31] Loading and sampling 34 raster layers for group 0eedd36ec93fd9c99387cc8f5c320801
[16:57:05] Loading and sampling 11 raster layers for group 1111cac5fd6669660413a5daaf395e08
[16:57:14] Loading and sampling 1 raster layers for group 20b4790b3a160a2ac8adefedfe07fbe7
[16:57:14] Loading and sampling 123 raster layers for group 2bff5e510f1e47be04d69728dd453b6b
[16:57:18] Loading and sampling 3 raster layers for group 2fbfe950e4c22a02ba7e3b52884be34f
[16:57:22] Loading and sampling 1 raster layers for group 435d45ba442271d360c4ea7ec0c92bc6
[16:57:23] Loading and sampling 7 raster layers for group 4febf6577f1f162c52ad482e075d0c76
[16:57:23] Loading and sampling 1 raster layers for group 56140668da1ff98e935fb06418a942f2
[16:57:27] Loading and sampling 139 raster layers for group 660960b8a39c6afeae9171be064ae057
[16:58:26] Loading and sampling 10 raster layers for group 67d1c8ae1f1ff92ae4147038afb08e2b
[16:58:33] Loading and sampling 1 raster la

Error 2: issues in reading the file with URL /vsicurl/http://192.168.1.31:8333/global/clm/clm_lst_mod11a2.daytime_p05_1km_s0..0cm_2007.11.01..2007.11.30_v1.2.tif


[17:37:05] Loading and sampling 46 raster layers for group d2d3edaee8fa7d225dd92d9cde49bb0a
[17:38:24] Loading and sampling 12 raster layers for group ea4394b71841b299979376c6d822b513
[17:38:25] Loading and sampling 10 raster layers for group ebe5ac182537081e79c259a3002b39fb
[17:38:31] Loading and sampling 9 raster layers for group f36bf6d46c61a893ba043e68eb7ede16
[17:38:32] Loading and sampling 2 raster layers for group fe57de8e36b7458f841fd3aa6e9f719a
[17:38:32] Running the overlay for 2008
[17:38:32] Loading and sampling 34 raster layers for group 0eedd36ec93fd9c99387cc8f5c320801
[17:39:04] Loading and sampling 11 raster layers for group 1111cac5fd6669660413a5daaf395e08
[17:39:14] Loading and sampling 1 raster layers for group 20b4790b3a160a2ac8adefedfe07fbe7
[17:39:15] Loading and sampling 123 raster layers for group 2bff5e510f1e47be04d69728dd453b6b
[17:39:18] Loading and sampling 3 raster layers for group 2fbfe950e4c22a02ba7e3b52884be34f
[17:39:22] Loading and sampling 1 raster la

Error 2: issues in reading the file with URL /vsicurl/http://192.168.1.43:8333/ai4sh-landmasked/savi/savi_glad.landsat.ard2.seasconv.bimonthly.m_30m_s_20090301_20090430_eu_epsg.3035_v20231218.tif


[17:45:40] Loading and sampling 1 raster layers for group 6df3e8a271732db3b9607012924e00d9
[17:45:42] Loading and sampling 1 raster layers for group 6ed680f0705ae492bb83f45e130f7980
[17:45:42] Loading and sampling 10 raster layers for group 725e180fc0b4da112983d7131869020a
[17:45:47] Loading and sampling 4 raster layers for group 741e70ee94219b346a230b59d312a20d
[17:45:48] Loading and sampling 1 raster layers for group 7c8a64ee2ec5950fa58ad3d6cffd6c64
[17:45:48] Loading and sampling 44 raster layers for group 82b14127aedf4840b02b3a63f38f9d3a
[17:46:22] Loading and sampling 6 raster layers for group 88cae960bc1da66c8a769730a4a4004b
[17:46:23] Loading and sampling 1 raster layers for group 907f01ccbcf02de8f030abad8719de2c
[17:46:23] Loading and sampling 12 raster layers for group 91dee81fa0680fbc27bfbe37a6cd2fc2
[17:46:25] Loading and sampling 1 raster layers for group 922fbfb1bd358ca87af1803a9da7ef6c
[17:46:26] Loading and sampling 5 raster layers for group a252e434b807da2663ef4849b1b74

Error 2: issues in reading the file with URL /vsicurl/http://192.168.1.40:8333/ai4sh-landmasked/evi/evi_glad.landsat.ard2.seasconv_m_30m_s_20200101_20200228_eu_epsg.3035_v20231127.tif


[18:43:47] Loading and sampling 1 raster layers for group 6df3e8a271732db3b9607012924e00d9
[18:43:50] Loading and sampling 1 raster layers for group 6ed680f0705ae492bb83f45e130f7980
[18:43:51] Loading and sampling 10 raster layers for group 725e180fc0b4da112983d7131869020a
[18:43:57] Loading and sampling 4 raster layers for group 741e70ee94219b346a230b59d312a20d
[18:43:57] Loading and sampling 1 raster layers for group 7c8a64ee2ec5950fa58ad3d6cffd6c64
[18:43:58] Loading and sampling 44 raster layers for group 82b14127aedf4840b02b3a63f38f9d3a
[18:44:32] Loading and sampling 6 raster layers for group 88cae960bc1da66c8a769730a4a4004b
[18:44:32] Loading and sampling 1 raster layers for group 907f01ccbcf02de8f030abad8719de2c
[18:44:33] Loading and sampling 12 raster layers for group 91dee81fa0680fbc27bfbe37a6cd2fc2
[18:44:34] Loading and sampling 1 raster layers for group 922fbfb1bd358ca87af1803a9da7ef6c
[18:44:34] Loading and sampling 5 raster layers for group a252e434b807da2663ef4849b1b74